In [1]:
import torch
import torch.multiprocessing as mp
import torch.nn as nn
from qpth.qp import QPFunction

import numpy as np
import random

import data_generator
import params_newsvendor as params

from sklearn.preprocessing import StandardScaler

from model import VariationalLayer, VariationalNet, StandardNet, VariationalNet2

from train import TrainDecoupled, TrainCombined

from train_normflow import TrainFlowDecoupled, TrainFlowCombined

import joblib


import pdb

In [2]:
cpu_count = mp.cpu_count()
is_cuda = False
dev = torch.device('cpu')  
if torch.cuda.is_available():
    is_cuda = True
    dev = torch.device('cuda')
    #cpu_count = 1

In [3]:
# Setting the seeds to allow replication
# Changing the seed might require hyperparameter tuning again
# Because it changes the deterministic parameters
seed_number = 0
np.random.seed(seed_number)
torch.manual_seed(seed_number)
random.seed(seed_number)

In [4]:
# Setting parameters (change if necessary)
N = 8000 # Total data size
N_train = 5000 # Training data size
N_SAMPLES = 2 # Sampling size while training
BATCH_SIZE_LOADER = 16 # Standard batch size
EPOCHS = 150 

In [5]:
nl = 1

In [6]:
# Data manipulation
N_valid = N - N_train
X, Y_original = data_generator.data_4to8(N_train, noise_level=nl)

In [7]:
# Output normalization
scaler = StandardScaler()
scaler.fit(Y_original)
tmean = torch.tensor(scaler.mean_)
tstd = torch.tensor(scaler.scale_)
joblib.dump(scaler, 'scaler_multi.gz')

['scaler_multi.gz']

In [8]:
def inverse_transform(yy):
    return yy*tstd + tmean

Y = scaler.transform(Y_original).copy()
X = torch.tensor(X, dtype=torch.float32)#.to(dev)
Y = torch.tensor(Y, dtype=torch.float32)#.to(dev)
Y_original = torch.tensor(Y_original, dtype=torch.float32)#.to(dev)

In [9]:
data_train_original = data_generator.ArtificialDataset(X, Y_original)
training_loader_original = torch.utils.data.DataLoader(
    data_train_original, batch_size=BATCH_SIZE_LOADER,
    shuffle=False, num_workers=cpu_count)
   
    
data_train = data_generator.ArtificialDataset(X, Y)
training_loader = torch.utils.data.DataLoader(
    data_train, batch_size=BATCH_SIZE_LOADER,
    shuffle=False, num_workers=cpu_count)
    
    
X_val, Y_val_original = data_generator.data_4to8(N_valid, noise_level=nl)
Y_val = scaler.transform(Y_val_original).copy()
X_val = torch.tensor(X_val, dtype=torch.float32)#.to(dev)
Y_val_original = torch.tensor(Y_val_original, dtype=torch.float32)#.to(dev)
Y_val = torch.tensor(Y_val, dtype=torch.float32)#.to(dev)


data_valid_original = data_generator.ArtificialDataset(X_val, Y_val_original)
validation_loader_original = torch.utils.data.DataLoader(
    data_valid_original, batch_size=BATCH_SIZE_LOADER,
    shuffle=False, num_workers=cpu_count)

    
data_valid = data_generator.ArtificialDataset(X_val, Y_val)
validation_loader = torch.utils.data.DataLoader(
    data_valid, batch_size=BATCH_SIZE_LOADER,
    shuffle=False, num_workers=cpu_count)
    
input_size = X.shape[1]
output_size = Y.shape[1]

In [10]:
class SolveNewsvendorWithKKT():
    def __init__(self, params_t, n_samples):
        super(SolveNewsvendorWithKKT, self).__init__()
        
        self.params_t = params_t
        
        n_items = len(params_t['c'])
        self.n_items = n_items  
        self.n_samples = n_samples
            
        # Torch parameters for KKT         
        ident = torch.eye(n_items)
        ident_samples = torch.eye(n_items*n_samples)
        ident3 = torch.eye(n_items + 2*n_items*n_samples)
        zeros_matrix = torch.zeros((n_items*n_samples, n_items*n_samples))
        zeros_array = torch.zeros(n_items*n_samples)
        ones_array = torch.ones(n_items*n_samples)
             
        self.Q = torch.diag(
            torch.hstack(
                (
                    params_t['q'], 
                    (1/n_samples)*params_t['qs'].repeat_interleave(n_samples), 
                    (1/n_samples)*params_t['qw'].repeat_interleave(n_samples)
                )
            )).to(dev)
        
        
        self.lin = torch.hstack(
                                (
                                    params_t['c'], 
                                    (1/n_samples)*params_t['cs'].repeat_interleave(n_samples), 
                                    (1/n_samples)*params_t['cw'].repeat_interleave(n_samples)
                                )).to(dev)
             
            
        shortage_ineq = torch.hstack(
            (
                -ident.repeat_interleave(n_samples, 0), 
                -ident_samples, 
                zeros_matrix
            )
        )  
        
        
        excess_ineq = torch.hstack(
            (
                ident.repeat_interleave(n_samples, 0), 
                zeros_matrix, 
                -ident_samples
            )
        )
        
        
        price_ineq = torch.hstack(
            (
                params_t['pr'], 
                zeros_array, 
                zeros_array
            )
        )
        
        
        positive_ineq = -ident3
        
        
        self.ineqs = torch.vstack(
            (
                shortage_ineq, 
                excess_ineq, 
                price_ineq, 
                positive_ineq
            )
        ).to(dev)
 
        self.uncert_bound = torch.hstack((-ones_array, ones_array)).to(dev)
        
        self.determ_bound = torch.tensor([params_t['B']]) 
        
        self.determ_bound = torch.hstack((self.determ_bound, 
                                          torch.zeros(n_items), 
                                          torch.zeros(n_items*n_samples), 
                                          torch.zeros(n_items*n_samples))).to(dev)
        
        
        
    def forward(self, y):
        """
        Applies the qpth solver for all batches and allows backpropagation.
        Formulation based on Priya L. Donti, Brandon Amos, J. Zico Kolter (2017).
        Note: The quadratic terms (Q) are used as auxiliar terms only to allow the backpropagation through the 
        qpth library from Amos and Kolter. 
        We will set them as a small percentage of the linear terms (Wilder, Ewing, Dilkina, Tambe, 2019)
        """
        
        batch_size, n_samples_items = y.size()
                
        assert self.n_samples*self.n_items == n_samples_items 

        Q = self.Q
        Q = Q.expand(batch_size, Q.size(0), Q.size(1))
        
        lin = self.lin
        lin = lin.expand(batch_size, lin.size(0))

        ineqs = torch.unsqueeze(self.ineqs, dim=0)
        ineqs = ineqs.expand(batch_size, ineqs.shape[1], ineqs.shape[2])       

        uncert_bound = (self.uncert_bound*torch.hstack((y, y)))
        determ_bound = self.determ_bound.unsqueeze(dim=0).expand(
            batch_size, self.determ_bound.shape[0])
        bound = torch.hstack((uncert_bound, determ_bound))     
        
        e = torch.DoubleTensor().to(dev)
        
        argmin = QPFunction(verbose=-1)\
            (Q.double(), lin.double(), ineqs.double(), 
             bound.double(), e, e).double()
            
        return argmin[:,:n_items]
    
    def cost_per_item(self, Z, Y):
        return params_t['q'].to(dev)*Z.to(dev)**2 \
            + self.params_t['qs'].to(dev)*(torch.max(torch.zeros((self.n_items)).to(dev),Y.to(dev)-Z.to(dev)))**2 \
            + self.params_t['qw'].to(dev)*(torch.max(torch.zeros((self.n_items)).to(dev),Z.to(dev)-Y.to(dev)))**2 \
            + self.params_t['c'].to(dev)*Z.to(dev) \
            + self.params_t['cs'].to(dev)*torch.max(torch.zeros((self.n_items)).to(dev),Y.to(dev)-Z.to(dev)) \
            + self.params_t['cw'].to(dev)*torch.max(torch.zeros((self.n_items)).to(dev),Z.to(dev)-Y.to(dev))

    
    def reshape_outcomes(self, y_pred):
                
        if len(y_pred.shape) == 2:
            y_pred = y_pred.unsqueeze(0)

        n_samples = y_pred.shape[0]
        batch_size = y_pred.shape[1]
        #n_items = y_pred.shape[2]
        y_pred = y_pred.permute((1, 2, 0)).reshape((batch_size, n_samples*self.n_items))
        return y_pred
    
    def calc_f_por_item(self, y_pred, y):
        #pdb.set_trace()
        y_pred = self.reshape_outcomes(y_pred)
        z_star =  self.forward(y_pred)
        f_per_item = self.cost_per_item(z_star, y)
        return f_per_item

    def calc_f_per_day(self, y_pred, y):
        f_per_item = self.calc_f_por_item(y_pred, y)
        f = torch.sum(f_per_item, 1)
        return f

    def end_loss(self, y_pred, y):
        f = self.calc_f_per_day(y_pred, y)
        f_total = torch.mean(f)
        return f_total
    
    def end_loss_dist(self, y_pred, y):
        f = self.calc_f_per_day(y_pred, y)
        f_total = torch.mean(f)
        return f_total

cost_per_item = lambda Z, Y : params_t['q'].to(dev)*Z.to(dev)**2 \
                            + params_t['qs'].to(dev)*(torch.max(torch.zeros((n_items)).to(dev),Y.to(dev)-Z.to(dev)))**2 \
                            + params_t['qw'].to(dev)*(torch.max(torch.zeros((n_items)).to(dev),Z.to(dev)-Y.to(dev)))**2 \
                            + params_t['c'].to(dev)*Z.to(dev) \
                            + params_t['cs'].to(dev)*torch.max(torch.zeros((n_items)).to(dev),Y.to(dev)-Z.to(dev)) \
                            + params_t['cw'].to(dev)*torch.max(torch.zeros((n_items)).to(dev),Z.to(dev)-Y.to(dev))


def reshape_outcomes(y_pred):
    n_samples = y_pred.shape[0]
    batch_size = y_pred.shape[1]
    n_items = y_pred.shape[2]

    y_pred = y_pred.permute((1, 2, 0)).reshape((batch_size, n_samples*n_items))

    return y_pred

def calc_f_por_item(y_pred, y):
    y_pred = reshape_outcomes(y_pred)
    z_star =  argmin_solver(y_pred)
    f_per_item = cost_per_item(z_star, y)
    return f_per_item

def calc_f_per_day(y_pred, y):
    f_per_item = calc_f_por_item(y_pred, y)
    f = torch.sum(f_per_item, 1)
    return f

def cost_fn(y_pred, y):
    f = calc_f_per_day(y_pred, y)
    f_total = torch.mean(f)
    return f_total

In [11]:
h_ann_dec = StandardNet(input_size, output_size, 0).to(dev)
h_bnn_dec = VariationalNet2(N_SAMPLES, input_size, output_size, 1.0).to(dev)

h_ann_com = StandardNet(input_size, output_size, 0).to(dev)
h_bnn_com = VariationalNet2(N_SAMPLES, input_size, output_size, 1.0).to(dev)

opt_h_ann_dec = torch.optim.Adam(h_ann_dec.parameters(), lr=0.001)
opt_h_bnn_dec = torch.optim.Adam(h_bnn_dec.parameters(), lr=0.001)

opt_h_ann_com = torch.optim.Adam(h_ann_com.parameters(), lr=0.005)
opt_h_bnn_com = torch.optim.Adam(h_bnn_com.parameters(), lr=0.005)

mse_loss = nn.MSELoss(reduction='none')

In [12]:
n_items = output_size
params_t, _ = params.get_params(n_items, seed_number)

# Construct the solver
cn_constrained = SolveNewsvendorWithKKT(params_t, 1)
cn_constrained_dist = SolveNewsvendorWithKKT(params_t, N_SAMPLES*4)

In [ ]:
train_ANN_dec = TrainDecoupled(
                    bnn = False,
                    model=h_ann_dec,
                    opt=opt_h_ann_dec,
                    loss_data=mse_loss,
                    K=0.0,
                    training_loader=training_loader,
                    validation_loader=validation_loader,
                    dev = dev
                )

train_ANN_dec.train(EPOCHS=EPOCHS)
model_ann_dec = train_ANN_dec.model

------------------EPOCH 1------------------
DATA LOSS 	 train 0.453 valid 0.331
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.45 valid 0.33
------------------EPOCH 2------------------
DATA LOSS 	 train 0.272 valid 0.269
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.27 valid 0.27
------------------EPOCH 3------------------
DATA LOSS 	 train 0.23 valid 0.247
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.23 valid 0.25
------------------EPOCH 4------------------
DATA LOSS 	 train 0.212 valid 0.235
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.21 valid 0.23
------------------EPOCH 5------------------


In [ ]:
train_ANN_com = TrainCombined(
                    bnn = False,
                    model=h_ann_com,
                    opt=opt_h_ann_com,
                    training_loader=training_loader_original,
                    validation_loader=validation_loader_original,
                    OP = cn_constrained,
                    dev = dev
                )

train_ANN_com.train(EPOCHS=35)
model_ann_com = train_ANN_com.model

In [ ]:
train_BNN_dec = TrainDecoupled(
                    bnn = True,
                    model=h_bnn_dec,
                    opt=opt_h_bnn_dec,
                    loss_data=mse_loss,
                    K=0.0,
                    training_loader=training_loader_original,
                    validation_loader=validation_loader_original
                )

train_BNN_dec.train(EPOCHS=EPOCHS)
model_bnn_dec = train_BNN_dec.model

In [ ]:
train_BNN_com = TrainCombined(
                    bnn = True,
                    model=h_bnn_com,
                    opt=opt_h_bnn_com,
                    training_loader=training_loader_original,
                    validation_loader=validation_loader_original,
                    OP = cn_constrained_dist
                )

train_BNN_com.train(EPOCHS=8)
model_bnn_com = train_BNN_com.model

In [ ]:
trfl = TrainFlowDecoupled(steps = 5000, input_size=4, output_size=8)
pyx = trfl.train(X, Y, X_val, Y_val)
model_flow_dec = pyx

In [ ]:
# Training regression with FLOW
trfl = TrainFlowCombined(
    steps = 250, 
    input_size=4,
    output_size=8,
    lr=8e-3, 
    OP = cn_constrained_dist,
    n_samples=N_SAMPLES*4)
pyx = trfl.train(X, Y, X_val, Y_val)
model_flow_com = pyx

In [ ]:
# Propagating predictions to Newsvendor Problem


Y_pred_ANN_dec = model_ann_dec(X_val).unsqueeze(0)
Y_pred_ANN_dec = inverse_transform(Y_pred_ANN_dec)

Y_pred_ANN_com = model_ann_com(X_val).unsqueeze(0)
Y_pred_ANN_com = inverse_transform(Y_pred_ANN_com)

M = 4
model_bnn_dec.update_n_samples(n_samples=M)
Y_pred_BNN_dec = train_BNN_dec.model.forward_dist(X_val)
Y_pred_BNN_dec = inverse_transform(Y_pred_BNN_dec)

model_bnn_com.update_n_samples(n_samples=M)
Y_pred_BNN_com = train_BNN_com.model.forward_dist(X_val)
Y_pred_BNN_com = inverse_transform(Y_pred_BNN_com)
M = Y_pred_BNN_com.shape[0]

N = X_val.shape[0]
Y_pred_flow_dec = torch.zeros((M, N, n_items))
for i in range(0, N):
    Y_pred_flow_dec[:,i,:] = model_flow_dec.condition(X_val[i]).sample(torch.Size([M,])).squeeze()
Y_pred_flow_dec = inverse_transform(Y_pred_flow_dec)

#Y_pred_flow_com = torch.zeros((M, N, n_items))
#for i in range(0, N):
#    Y_pred_flow_com[:,i,:] = model_com.condition(X_val[i]).sample(torch.Size([M,])).squeeze()
#Y_pred_flow_com = inverse_transform(Y_pred_flow_com)

In [ ]:
mse_loss = nn.MSELoss()
print(mse_loss(Y_pred_ANN_dec.mean(axis=0), Y_val_original))
print(mse_loss(Y_pred_ANN_com.mean(axis=0), Y_val_original))

print(mse_loss(Y_pred_BNN_dec.mean(axis=0), Y_val_original))
print(mse_loss(Y_pred_BNN_com.mean(axis=0), Y_val_original))

print(mse_loss(Y_pred_flow_dec.mean(axis=0), Y_val_original))
#print(mse_loss(Y_pred_flow_com.mean(axis=0), Y_val_original))

In [ ]:
# Construct the solver
newsvendor_solve_kkt = SolveNewsvendorWithKKT(params_t, 1)
newsvendor_solve_kkt_M = SolveNewsvendorWithKKT(params_t, M)

In [ ]:
def optimize_final_block(Y_pred, cost):

    n_batches = int(np.ceil(Y_pred.shape[1]/BATCH_SIZE_LOADER))

    f_total = 0
    f_total_best = 0

    for b in range(0, n_batches):
        i_low = b*BATCH_SIZE_LOADER
        i_up = (b+1)*BATCH_SIZE_LOADER
        if b == n_batches-1:
            i_up = n_batches*Y_pred.shape[1]
        f_total += cost(Y_pred[:,i_low:i_up,:], Y_val_original[i_low:i_up,:])/n_batches
        print(b, f_total)

In [ ]:
optimize_final_block(Y_pred_ANN_dec, newsvendor_solve_kkt.end_loss)

In [ ]:
optimize_final_block(Y_pred_ANN_com, newsvendor_solve_kkt.end_loss)

In [ ]:
optimize_final_block(Y_pred_BNN_dec, newsvendor_solve_kkt_M.end_loss_dist)

In [ ]:
optimize_final_block(Y_pred_BNN_com, newsvendor_solve_kkt_M.end_loss_dist)

In [ ]:
optimize_final_block(Y_pred_flow_dec, newsvendor_solve_kkt_M.end_loss_dist)

In [ ]:
def argmin_solver(Y_pred):
            z_star = newsvendor_solve_kkt_M.forward(Y_pred)
            return z_star

optimize_final_block(Y_pred_BNN_dec, True)

In [ ]:
optimize_final_block(Y_pred_BNN_com, True)

In [ ]:
optimize_final_block(Y_pred_flow_dec, True)

In [ ]:
optimize_final_block(Y_pred_flow_com, True)

In [ ]:
def argmin_solver(y_pred):
    z_star = newsvendor_solve_kkt.forward(y_pred)
    return z_star

n_batches = int(np.ceil(Y_pred_ANN_dec.shape[1]/BATCH_SIZE_LOADER))

f_total = 0
f_total_best = 0

for b in range(0, n_batches):
    i_low = b*BATCH_SIZE_LOADER
    i_up = (b+1)*BATCH_SIZE_LOADER
    if b == n_batches-1:
        i_up = n_batches*Y_pred_ANN_dec.shape[1]
    f_total += cost_fn(Y_pred_ANN_dec[:,i_low:i_up,:], Y_val_original[i_low:i_up,:])/n_batches
    print(b, f_total)

In [ ]:
def argmin_solver(y_pred):
    z_star = newsvendor_solve_kkt_M.forward(y_pred)
    return z_star

n_batches = int(np.ceil(Y_pred_BNN.shape[1]/BATCH_SIZE_LOADER))

f_total = 0
f_total_best = 0

for b in range(0, n_batches):
    i_low = b*BATCH_SIZE_LOADER
    i_up = (b+1)*BATCH_SIZE_LOADER
    if b == n_batches-1:
        i_up = n_batches*Y_pred_BNN.shape[1]
    f_total += cost_fn(Y_pred_BNN[:,i_low:i_up,:], Y_val_original[i_low:i_up,:])/n_batches
    print(b, f_total)

In [ ]:
def argmin_solver(y_pred):
    z_star = newsvendor_solve_kkt_M.forward(y_pred)
    return z_star

n_batches = int(np.ceil(Y_pred_flow.shape[1]/BATCH_SIZE_LOADER))

f_total = 0
f_total_best = 0

for b in range(0, n_batches):
    i_low = b*BATCH_SIZE_LOADER
    i_up = (b+1)*BATCH_SIZE_LOADER
    if b == n_batches-1:
        i_up = n_batches*Y_pred_flow.shape[1]
    f_total += cost_fn(Y_pred_flow[:,i_low:i_up,:], Y_val_original[i_low:i_up,:])/n_batches
    print(b, f_total)

In [ ]:
Y_val_original.shape

In [ ]:
def argmin_solver(y_pred):
    z_star = newsvendor_solve_kkt.forward(y_pred)
    return z_star

n_batches = int(np.ceil(Y_val_original.shape[0]/BATCH_SIZE_LOADER))

f_total = 0
f_total_best = 0

for b in range(0, n_batches):
    i_low = b*BATCH_SIZE_LOADER
    i_up = (b+1)*BATCH_SIZE_LOADER
    if b == n_batches-1:
        i_up = n_batches*Y_val_original.shape[0]
    f_total += cost_fn(Y_val_original[i_low:i_up,:].unsqueeze(0), Y_val_original[i_low:i_up,:])/n_batches
    print(f_total)

In [ ]:
(argmin_solver(reshape_outcomes(Y_val_original[0:50,:].unsqueeze(0)))*params_t['pr']).sum(1)

1.83 1.64 2.43 28104 27870 27787 13598
1.87 1.89 2.46 42607 42070 41574 28124
1.80 1.85 2.25 34329 33855 33395 15849


